# scrapy中DownloadMiddlewera用法

Downloader Middleware是下载中间件, 处理过程主要在调度器发送requests请求的时候以及网页将response结果返回给spiders的时候，下载中间件是介于Scrapy的request/response处理的钩子，用于修改Scrapy request和response。

### 编写自己的下载中间件

编写下载器中间件，需要定义以下一个或者多个方法的python类

为了演示中间件的使用方法，这里创建一个项目: 爬取httpbin.org这个网站
```
scrapy startproject httpbintest 
cd httpbintest
scrapy genspider httpbin httpbin.com
```

创建好爬虫之后我们讲httpbin.py中的parse方法改成：
```python
def parse(self, response):
    print(response.text)
```    

然后通过命令行启动爬虫：`scrapy crawl httpbin`

现在通过代理中间件来实现ip的伪装,在middleares.py中写如下的中间件类：
```python
class ProxyMiddleare(object):
    logger = logging.getLogger(__name__)
    def process_request(self,request, spider):
        self.logger.debug("Using Proxy")
        request.meta['proxy'] = 'http://127.0.0.1:9743'
        return None
```    

然后在settings.py配置文件中开启下载中间件的功能，默认是关闭的
```
DOWNLOADER_MIDDLEWARES = {
   #'httpbintest.middlewares.HttpbintestDownloaderMiddleware': 543,
   'httpbintest.middlewares.ProxyMiddleware': 543
}
```

然后我们再次启动爬虫：`scrapy crawl httpbin`,

从日志中可以看出定义的中间件已经启动，并且输入了打印的日志信息，并且查看origin的ip地址也已经成了代理的ip地址，这样代理中间件成功了.

### 详细说明

### `class Scrapy.downloadermiddleares.DownloaderMiddleware`
##### `process_request(request,spider)`

当每个request通过下载中间件时，`process_request`方法被调用，该方法必须返回以下三种中的任意一种, 三种返回值的作用是不同的。
1. None,
2. 返回一个Response对象，
3. 返回一个Request对象或raise IgnoreRequest。

None:Scrapy将继续处理该request，执行其他的中间件的相应方法，直到合适的下载器处理函数(download handler)被调用,该request被执行(其response被下载)。

Response对象：Scrapy将不会调用任何其他的process_request()或process_exception() 方法，或相应地下载函数；其将返回该response。 已安装的中间件的 process_response() 方法则会在每个response返回时被调用。

Request对象：Scrapy则停止调用 process_request方法并重新调度返回的request。当新返回的request被执行后， 相应地中间件链将会根据下载的response被调用。

raise一个IgnoreRequest异常：则安装的下载中间件的 process_exception() 方法会被调用。如果没有任何一个方法处理该异常， 则request的errback(Request.errback)方法会被调用。如果没有代码处理抛出的异常， 则该异常被忽略且不记录。

#### `process_response(request, response, spider)`

process_response的返回值也是有三种：
1. response对象，
2. request对象，
3. raise一个IgnoreRequest异常

如果其返回一个Response(可以与传入的response相同，也可以是全新的对象)， 该response会被在链中的其他中间件的 process_response() 方法处理。

如果其返回一个 Request 对象，则中间件链停止， 返回的request会被重新调度下载。处理类似于 process_request() 返回request所做的那样。

如果其抛出一个 IgnoreRequest 异常，则调用request的errback(Request.errback)。 如果没有代码处理抛出的异常，则该异常被忽略且不记录(不同于其他异常那样)。

这里我们写一个简单的例子还是上面的项目，我们在中间件中继续添加如下代码：

```python
class ProxyMiddleware(object):
    logger = logging.getLogger(__name__)

    def process_request(self, request, spider):
        self.logger.debug('using proxy')
        request.meta['proxy'] = 'http://127.0.0.1:9743'
        return None

    def process_response(self, request, response, spider):
        response.status = 201
        return response
```

然后在spider中打印状态码：输出的状态码为201

#### `process_exception(request, exception, spider)`

当下载处理器(download handler)或 process_request() (下载中间件)抛出异常(包括 IgnoreRequest 异常)时，Scrapy调用 process_exception()。

process_exception() 也是返回三者中的一个: 
1. 返回 None 
2. 一个 Response 对象
3. 一个 Request 对象

如果其返回 None ，Scrapy将会继续处理该异常，接着调用已安装的其他中间件的 `process_exception()` 方法，直到所有中间件都被调用完毕，则调用默认的异常处理。

如果其返回一个 Response 对象，则已安装的中间件链的 `process_response()` 方法被调用。Scrapy将不会调用任何其他中间件的 `process_exception()` 方法。

如果其返回一个 Request 对象， 则返回的request将会被重新调用下载。这将停止中间件的 `process_exception()`方法执行，就如返回一个response的那样。 这个是非常有用的，就相当于如果我们失败了可以在这里进行一次失败的重试，例如当我们访问一个网站出现因为频繁爬取被封ip就可以在这里设置增加代理继续访问.